网络爬取之前的贮备工作
    检查robet.txt文件
    检查sitemap文件内容
    估算网站大小（渐变方法是通过google搜索site：+网络地址的方法来粗略确认网站的大小）
    识别网站使用的技术（builtwith.parse(网络主机地址)）可以显示网站的信息,见下面的例子
    寻找网站所有者（通过whois模块）
    

In [1]:
#设定固定的文件夹
in_path='e:/material/'
out_path='e:/result/'

In [26]:
#设定多个agent
import sqlite3 as sq
import urllib.request as ul
import requests as rq
import urllib
from bs4 import BeautifulSoup as bl
import random
import re
import pandas as pd
auto_headers = ["Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:30.0) Gecko/20100101 Firefox/30.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.75.14 (KHTML, like Gecko) Version/7.0.3 Safari/537.75.14",
    "Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; Win64; x64; Trident/6.0)",
    'Mozilla/5.0 (Windows; U; Windows NT 5.1; it; rv:1.8.1.11) Gecko/20071127 Firefox/2.0.0.11',
    'Opera/9.25 (Windows NT 5.1; U; en)',
    'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)',
    'Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.5 (like Gecko) (Kubuntu)',
    'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.8.0.12) Gecko/20070731 Ubuntu/dapper-security Firefox/1.5.0.12',
    'Lynx/2.8.5rel.1 libwww-FM/2.14 SSL-MM/1.4.1 GNUTLS/1.2.9',
    "Mozilla/5.0 (X11; Linux i686) AppleWebKit/535.7 (KHTML, like Gecko) Ubuntu/11.04 Chromium/16.0.912.77 Chrome/16.0.912.77 Safari/535.7",
    "Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:10.0) Gecko/20100101 Firefox/10.0",
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36']

global headers
headers = {'User-Agent': random.choice(auto_headers)}

In [11]:
old_url = 'https://www.sosody.net/v/80806-2-0.html'
req = ul.Request(old_url)
response = ul.urlopen(req)
print ('Old_url:' + old_url)
print ('Real_url:' + response.geturl())

Old_url:https://www.sosody.net/v/80806-2-0.html
Real_url:https://www.sosody.net/v/80806-2-0.html


In [14]:
import builtwith
import whois
website='https://www.qunar.com/'
data=builtwith.parse(website)
for i in data:
    print(data[i])
print('\n\n\n')
data=whois.whois(website)
for i in data:
    print(data[i])

['ClickHeat']
['PHP']
['Handlebars', 'jQuery']




['QUNAR.COM', 'qunar.com']
MarkMonitor, Inc.
whois.markmonitor.com
None
[datetime.datetime(2016, 1, 11, 15, 40, 19), datetime.datetime(2016, 1, 11, 7, 43, 30)]
[datetime.datetime(2005, 5, 9, 8, 43, 11), datetime.datetime(2005, 5, 9, 1, 43, 11)]
[datetime.datetime(2022, 5, 9, 8, 43, 11), datetime.datetime(2022, 5, 9, 1, 43, 11)]
['NS1.QUNAR.COM', 'NS3.QUNAR.COM', 'NS4.QUNAR.COM', 'NS5.QUNAR.COM', 'NS6.QUNAR.COM', 'NS8.QUNAR.COM', 'NS9.QUNAR.COM', 'ns9.qunar.com', 'ns4.qunar.com', 'ns6.qunar.com', 'ns5.qunar.com', 'ns1.qunar.com', 'ns8.qunar.com', 'ns3.qunar.com']
['clientDeleteProhibited https://icann.org/epp#clientDeleteProhibited', 'clientTransferProhibited https://icann.org/epp#clientTransferProhibited', 'clientUpdateProhibited https://icann.org/epp#clientUpdateProhibited', 'serverDeleteProhibited https://icann.org/epp#serverDeleteProhibited', 'serverTransferProhibited https://icann.org/epp#serverTransferProhibited', 'serverUpdatePro

编写第一个网络爬虫

学习设置网络代理

In [15]:
import json
def download(website,num_retries=3):#增加自动重新下载的参数num_retries，默认数量为3,user_agent为设置代理
    global headers
    print("Downloading:",website)
    request=ul.Request(website,headers=headers)
    try:
        html=ul.urlopen(website).read()
    except ul.URLError as e:
        print("download Error:",e.reason)
        html=None
        if hasattr(e,'code') and 500<=e.code<=600:
            return(download(website,num_retries-1))
    return (html)
website='http://www.xbiquge.la/10/10489/9683462.html'
html=bl(download(website),'lxml')
infors=html.find_all('div',id_='content')
print(infors)

Downloading: http://www.xbiquge.la/10/10489/9683462.html
[]


学习使用遍历爬虫

In [16]:
import itertools as it
for page in it.count(1):
    url='http://example.webscraping.com/view/-%d'% page
    html=download(url)
    if html is None:
        break
    else:
        print(url)
        print('\n')

Downloading: http://example.webscraping.com/view/-1
http://example.webscraping.com/view/-1


Downloading: http://example.webscraping.com/view/-2
http://example.webscraping.com/view/-2


Downloading: http://example.webscraping.com/view/-3
http://example.webscraping.com/view/-3


Downloading: http://example.webscraping.com/view/-4
http://example.webscraping.com/view/-4


Downloading: http://example.webscraping.com/view/-5
http://example.webscraping.com/view/-5


Downloading: http://example.webscraping.com/view/-6
http://example.webscraping.com/view/-6


Downloading: http://example.webscraping.com/view/-7
http://example.webscraping.com/view/-7


Downloading: http://example.webscraping.com/view/-8
http://example.webscraping.com/view/-8


Downloading: http://example.webscraping.com/view/-9
http://example.webscraping.com/view/-9


Downloading: http://example.webscraping.com/view/-10
http://example.webscraping.com/view/-10


Downloading: http://example.webscraping.com/view/-11
http://example.

In [ ]:
用来爬取链接的简单爬虫

In [17]:
import re
def get_links(html):
    webpage_regex=re.compile('<a[^>]+href=["\'](.*?)["\']',re.IGNORECASE)
    return(webpage_regex.findall(html))

def link_crawler(seed_url,link_regex):
    crawl_queue=[seed_url]
    while crawl_queue:
        url=crawl_queue.pop()
        html=download(url)
        for link in get_links(html):
            if re.match(link_regex,link):
                crawl_queue.append(link)
    list=set(crawl_queue)
    print(list)

link_crawler("http://example.webscraping.com",'/(index/view)')

Downloading: http://example.webscraping.com
download Error: TOO MANY REQUESTS


TypeError: expected string or bytes-like object

In [18]:
import sys

def ip_search(ip):
    host = 'http://saip.market.alicloudapi.com'
    path = '/ip'
    method = 'GET'
    appcode = '91fa754784494761b450672aba2d3efb'
    querys = 'ip='+ip
    bodys = {}
    url = host + path + '?' + querys
    head={'Authorization':'APPCODE '+appcode}
    request = ul.Request(url,headers=head)
    response = ul.urlopen(request)
    content = response.read()
    info=json.loads(content)
    return(info)
ip=input('请输入需要查询的IP地址:')
info=ip_search(ip)
k=info['showapi_res_body']
print('IP地址对应的信息如下:'+'\n')
for i in k:
    print(i,k[i])
c=input('请按任意键结束查询。。。。。。')

请输入需要查询的IP地址:192.168.0.1


HTTPError: HTTP Error 555: 555

In [19]:
url='https://www.mzitu.com/xinggan/page/2/'
global headers
req=rq.get(url,headers=headers)
html=req.text
#print(html)
uo_list=re.findall('<li><a(.*?)span></li>',html)
for i in uo_list:
    print(re.findall('https://www.mzitu.com/......',i)[0])

https://www.mzitu.com/209236
https://www.mzitu.com/225312
https://www.mzitu.com/209904
https://www.mzitu.com/209835
https://www.mzitu.com/209421
https://www.mzitu.com/224888
https://www.mzitu.com/205635
https://www.mzitu.com/209159
https://www.mzitu.com/205848
https://www.mzitu.com/224623
https://www.mzitu.com/208935
https://www.mzitu.com/224497
https://www.mzitu.com/224097
https://www.mzitu.com/208689
https://www.mzitu.com/205912
https://www.mzitu.com/224042
https://www.mzitu.com/224173
https://www.mzitu.com/205359
https://www.mzitu.com/223616
https://www.mzitu.com/223985
https://www.mzitu.com/216244
https://www.mzitu.com/223776
https://www.mzitu.com/205512
https://www.mzitu.com/223504


In [3]:
#制造首页地址的汇总
filename='pricepage.txt'
out_file=in_path+filename
count=1         
with open(out_file,'a') as tar:
    for i in range(1,51):
        link='http://www.100ppi.com/mprice/mlist-'+str(count)+'.html\n'
        tar.write(link)
        count+=1

In [9]:
linkname='link2.txt'
final_name=in_path+linkname
def get_pic_page(url):
    global headers
    req=rq.get(url,headers=headers,timeout=30)
    html=req.text
    uo_list=re.findall('<li><a(.*?)span></li>',html)
    with open(final_name,'a') as tar:
        count=1
        for i in uo_list:
            tar.write(re.findall('https://www.mzitu.com/......',i)[0]+'\n')
            count+=1
    

In [10]:
with open(out_file,'r') as sou:
    count=1
    for eachline in sou:
        get_pic_page(eachline.strip('\n'))
        count+=1
print(count)

ConnectionError: HTTPSConnectionPool(host='www.mzitu.com', port=443): Max retries exceeded with url: /xinggan/page/2/ (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000000007CC8AC8>: Failed to establish a new connection: [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。'))

In [ ]:
#抓取彩票数据
url=input('请输入双色球历史数据的网络地址:')


In [ ]:
global headers
req=rq.get(url,headers=headers)
print(req.text)

In [ ]:
html=bl(req.text,'lxml')

In [ ]:
summery_data=html.find_all('tr',class_="t_tr1")
print(summery_data)

In [ ]:
data_list=[]
for i in summery_data:
    caipiao={}
    caipiao['彩票期数']=i.find_all('td')[0].text
    caipiao['红球数字1']=i.find_all('td',class_="t_cfont2")[0].text
    caipiao['红球数字2']=i.find_all('td',class_="t_cfont2")[1].text
    caipiao['红球数字3']=i.find_all('td',class_="t_cfont2")[2].text
    caipiao['红球数字4']=i.find_all('td',class_="t_cfont2")[3].text
    caipiao['红球数字5']=i.find_all('td',class_="t_cfont2")[4].text
    caipiao['红球数字6']=i.find_all('td',class_="t_cfont2")[5].text
    caipiao['篮球数字']=i.find_all('td',class_="t_cfont4")[0].text
    data_list.append(caipiao)
for i in data_list:
    print(i)

In [6]:
#安居客二手房数据爬取
filename='koudaikorea.txt'
out_file=in_path+filename
count=1         
with open(out_file,'a') as tar:
    for i in range(1,104):
        link='https://www.kdys.cc/search.php?page='+str(count)+'&searchtype=5&tid=28&area=%E9%9F%A9%E5%9B%BD\n'
        tar.write(link)
        count+=1

In [ ]:
#生意社最细价格信息抓取和汇总，设计的流程和响应的函数如下
##获得对应的页面地址，http://www.100ppi.com/mprice/mlist-51.html（从1-50）------已经完成，输出文件pricepage-txt
##编写函数抓取对应页面的价格信息，函数名称price_get(url):
###需要使用到的库 requests，time ，os，bs4,random,re------已经加载
####需要使用随机的user_agent。------已经有随机的user-agent
###如果当前页面抓取失败，需要把抓取失败的页面的信息保存到errorlog里面，方便后面重新抓取
##每次抓取一个页面的信息，就讲对应的信息汇总病保存到1个excel数据表里面，表的名称用page 代表
##完成所有页面抓取后，汇总所有的数据表的价格信息并输出到一个数据表里面
##数据结构包含6个项目 ：品种名称 品牌与规格 报价 报价提供方 发布时间

In [ ]:
urls='http://www.100ppi.com/mprice/mlist-1.html'
def price_get(url):
    global headers
    price_list=[]
    dic={}
    req=rq.get(url,headers=headers,timeout=30)
    html=bl(req.text,'lxml')
    loc=html.find_all('tr')
    del loc[0]
    for i in loc:
        dic['品种名称']=i.find('td',class_="p-name a-blue")[0].text.replace('\n','').replace('\t','').replace('\r','')
        dic['品牌与规格']=i.find_all('td')[1].attrs['title']
        dic['报价']=i.find_all('td')[2].text.replace('\n','').replace('\t','').replace('\r','')
        dic['报价提供方']=i.find_all('td')[3].text.replace('\n','').replace('\t','').replace('\r','')
        dic['发布时间']=i.find_all('td')[4].text.replace('\n','').replace('\t','').replace('\r','')
        price_list.append(dic)
    print(len(loc))
price_get(url=urls)

In [3]:
origin_url='https://mm.enterdesk.com/3.html'
global headers
req=rq.get(origin_url,headers=headers)
print(req.text)

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"
        "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
<head>
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8"/>
    <title>美女图片_美女壁纸_mm图片_美女写真图片大全-回车桌面</title>
    <meta name="keywords" content="美女,美女图片,美女写真,美女壁纸,mm图片,美女图片大全">
    <meta name="description" content="回车美女图片,美女壁纸,mm图片,美女图片大全,汇集了大量清纯,性感,欧美,日韩美女写真图片,是国内最大最全的美女图片大全网站。">
    <script src="/static/ajax/libs/jquery/1.12.4/jquery.min.js"></script>
    <script src="/static/ajax/libs/masonry/4.2.0/masonry.pkgd.min.js"></script><script src="/static/ajax/libs/jquery.imagesloaded/4.1.3/imagesloaded.min.js"></script>
    <script src="/static/com.enter/2017/public.js?20180517"></script>
    <link href="/static/com.enter/2017/public.css" rel="stylesheet" type="text/css"/>
    <script src="/static/ajax/libs/layer/3.0.3/layer.min.js"></script>
    <script>browserRedirect("https://m.mm.enterdesk.com/3.

In [13]:
level_2_url=[]
html=bl(req.text,'lxml')
a_list=html.find_all('dd')
for i in a_list:
    level_2_url.append(i.find_all('a')[0].attrs['href'])
for i in level_2_url:
    print(i)
#print(html)

https://mm.enterdesk.com/bizhi/56508.html
https://mm.enterdesk.com/bizhi/56498.html
https://mm.enterdesk.com/bizhi/56495.html
https://mm.enterdesk.com/bizhi/56426.html
https://mm.enterdesk.com/bizhi/56423.html
https://mm.enterdesk.com/bizhi/56420.html
https://mm.enterdesk.com/bizhi/56324.html
https://mm.enterdesk.com/bizhi/56321.html
https://mm.enterdesk.com/bizhi/56318.html
https://mm.enterdesk.com/bizhi/56315.html
https://mm.enterdesk.com/bizhi/56312.html
https://mm.enterdesk.com/bizhi/56225.html
https://mm.enterdesk.com/bizhi/56204.html
https://mm.enterdesk.com/bizhi/56201.html
https://mm.enterdesk.com/bizhi/56198.html
https://mm.enterdesk.com/bizhi/56117.html


In [25]:
global headers
for url in level_2_url:
    req2=rq.get(url,headers=headers)
    html2=bl(req2.text,'lxml')
    #print(html2)
    pic_list=[]
    temp_list=html2.find_all('div',class_="swiper-slide")
    for i in temp_list:
        pic_list.append(i.find_all('img')[0].attrs['src'])
    for i in set(pic_list):
        print (i)

https://up.enterdesk.com/edpic_360_360/2d/a1/3a/2da13ac8f4616a1e01d174363b75c2a5.jpg
https://up.enterdesk.com/edpic_360_360/16/07/41/160741cf9d08fd4684e1178da65ddc41.jpg
https://up.enterdesk.com/edpic_360_360/bf/9d/02/bf9d02ecba829c57c4cfd0dc1e7ae3d2.jpg
https://up.enterdesk.com/edpic_360_360/41/a0/36/41a03643e7a74b2239fb981bfd4ca5e1.jpg
https://up.enterdesk.com/edpic_360_360/6b/5d/cd/6b5dcd16df1a573e2e536f9505ac1653.jpg
https://up.enterdesk.com/edpic_360_360/10/da/3e/10da3e98fe8933c59bb8fe6e039b4604.jpg
https://up.enterdesk.com/edpic_360_360/c7/39/a9/c739a9e2540742a193f29481a0366ad1.jpg
https://up.enterdesk.com/edpic_360_360/58/f2/30/58f230e45cb75aa89e4aa5ee00c39583.jpg
https://up.enterdesk.com/edpic_360_360/64/c4/c9/64c4c910a0ae8581c9de84084ca9e9fb.jpg
https://up.enterdesk.com/edpic_360_360/da/c1/58/dac158e76d1d6d58e29e2b33df517741.jpg
https://up.enterdesk.com/edpic_360_360/b5/85/e6/b585e678fa01b6e4368ac505fffeb0af.jpg
https://up.enterdesk.com/edpic_360_360/e3/db/d1/e3dbd1b03640fcc42

https://up.enterdesk.com/edpic_360_360/7d/8b/4c/7d8b4c780b34043d9a2e35310124a1cc.jpg
https://up.enterdesk.com/edpic_360_360/22/f9/d5/22f9d57b9b565897b0c14b8c1e80f6ca.jpg
https://up.enterdesk.com/edpic_360_360/a7/20/27/a7202748705174625be09f15b9db2368.jpg
https://up.enterdesk.com/edpic_360_360/50/ec/33/50ec33582e41d1e2d992cfa9dabf28e5.jpg
https://up.enterdesk.com/edpic_360_360/0b/f6/7c/0bf67c0eca84527e070ab422ab7f494a.jpg
https://up.enterdesk.com/edpic_360_360/64/b1/b7/64b1b7c68601bd2bcd104384007343d6.jpg
https://up.enterdesk.com/edpic_360_360/a3/93/b9/a393b9a48233f9698fc95d83b7fdcc2f.jpg
https://up.enterdesk.com/edpic_360_360/79/1f/90/791f903ad86b13e70c0dd0b81fddf296.jpg
https://up.enterdesk.com/edpic_360_360/ef/b3/d6/efb3d69e47472efbb68807e6e334a202.jpg
https://up.enterdesk.com/edpic_360_360/56/66/ff/5666fff9b21ace1779a3929d5528f210.jpg
https://up.enterdesk.com/edpic_360_360/df/83/dc/df83dc61316b81034e7260a5c5c52805.jpg
https://up.enterdesk.com/edpic_360_360/cc/e1/34/cce1342a440d5c836

In [27]:
db_name=out_path+input('请输入数据库的名称:')+'.db'
dx=sq.connect(db_name)

请输入数据库的名称:pic


In [28]:
cu=dx.cursor()
cu.execute('create table pic_rec(html_address primary key varchar(255) UNIQUE)')
dx.commit()

OperationalError: near "varchar": syntax error